<a href="https://colab.research.google.com/github/WesleySantosMaxtelll/doutorado/blob/main/lstm_setembroBR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install torch
!pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 32.6 MB/s 
     |████████████████████████████████| 86 kB 7.0 MB/s 
     |████████████████████████████████| 6.6 MB 65.7 MB/s 
     |████████████████████████████████| 596 kB 77.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import matplotlib.pyplot as plt
from pathlib import Path
import torch
import os
import torch.nn.functional as F
import numpy as np
import torch.nn as nn
from torch.optim import AdamW, SGD
from transformers import BertTokenizer, BertModel, BertForSequenceClassification, BertConfig
from torch.nn.utils.rnn import pack_padded_sequence
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.utils import class_weight
from collections import Counter
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import seaborn as sns
import time
from datetime import datetime
import gc
from torch.nn import CrossEntropyLoss

In [ ]:
import pickle
import random

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import svm
from sklearn.linear_model import LogisticRegression
import numpy as np
import pandas as pd
from sklearn.svm import OneClassSVM
import nltk
from nltk.corpus import stopwords
from numpy import asarray
# nltk.download('stopwords')
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import re

PATH_CORPUS = "/content/drive/MyDrive/doutorado/dados_v6/"


def get_data_full():
    f = pickle.load(open(PATH_CORPUS + '/depress-control_v3.pkl', 'rb'))
    return f.Text, f.Diagnosed_YN.apply(lambda x: 1 if x == 'yes' else -1)


def get_data_BERT(task):
    f = pickle.load(open(PATH_CORPUS + f'/{task}-control_v3.pkl', 'rb'))
    return f.User_ID, f.Text, f.Diagnosed_YN.apply(lambda x: 1 if x == 'yes' else -1)


def get_data_BERT_one_round(task):
    f_train = pickle.load(open(PATH_CORPUS + f'/train_{task}-control_v3.pkl', 'rb'))
    text_train, label_train =  f_train.Text, f_train.Diagnosed_YN.apply(lambda x: 1 if x == 'yes' else 0)
    f_test = pickle.load(open(PATH_CORPUS + f'/test_{task}-control_v3.pkl', 'rb'))
    text_test, label_test = f_test.Text, f_test.Diagnosed_YN.apply(lambda x: 1 if x == 'yes' else 0)
    return text_train, label_train, f_train.User_ID, text_test, label_test, f_test.User_ID


def get_data_one_round(task, data_type=''):
    d = {'depress': 'D', 'anxiety':"A"}
    if data_type == 'nomention':
        f_train = pickle.load(open(PATH_CORPUS + f'_no_mention/train_{task}-control_v3.pkl', 'rb'))
        text_train, label_train = f_train.Text, f_train.Diagnosed_YN.apply(lambda x: 1 if x == 'yes' else -1)
        f_test = pickle.load(open(PATH_CORPUS + f'/test_{task}-control_v3.pkl', 'rb'))
        text_test, label_test = f_test.Text, f_test.Diagnosed_YN.apply(lambda x: 1 if x == 'yes' else -1)
        return text_train, label_train, text_test, label_test
    else:
        f_train = pickle.load(open(PATH_CORPUS + f'/train_{d[task]}.pkl', 'rb'))
        text_train, label_train =  f_train.Text, f_train.Diagnosed_YN.apply(lambda x: 1 if x == 'yes' else -1)
        text_train = text_train.apply(lambda x: x.split('#'))
        f_test = pickle.load(open(PATH_CORPUS + f'/test_{d[task]}.pkl', 'rb'))
        text_test, label_test = f_test.Text, f_test.Diagnosed_YN.apply(lambda x: 1 if x == 'yes' else -1)
        text_test = text_test.apply(lambda x: x.split('#'))
        return text_train, label_train, text_test, label_test


def get_data(task):
    f = pickle.load(open(PATH_CORPUS + f'/train_{task}-control_v3.pkl', 'rb'))
    return f.Text, f.Diagnosed_YN.apply(lambda x: 1 if x == 'yes' else -1)
    # X_train, y_train, X_val, y_val, X_test, y_test = [], [], [], [], [], []
    # for k in f['val_text'].keys():
    #     X_val.append(f['val_text'][k])
    #     y_val.append(f['val_label'][k])
    # for k in f['train_text'].keys():
    #     X_train.append(f['train_text'][k])
    #     y_train.append(f['train_label'][k])
    # for k in f['test_text'].keys():
    #     X_test.append(f['test_text'][k])
    #     y_test.append(f['test_label'][k])
    # return X_train, y_train, X_val, y_val, X_test, y_test


# load embedding as a dict
def load_embedding(filename):
    # load embedding into memory, skip first line
    file = open(filename, 'r')
    lines = file.readlines()[1:]
    file.close()
    # create a map of words to vectors
    embedding = dict()
    for line in lines:
        parts = line.split()
        if len(parts) > 51:
            continue
        # key is string word, value is numpy array for vector
        embedding[parts[0]] = asarray([i.replace(',','.') for i in parts[1:]], dtype='float32')
    return embedding


def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]


def unison_shuffled_3_copies(a, b, c):
    assert len(a) == len(b)
    assert len(a) == len(c)
    p = np.random.permutation(len(a))
    return a[p], b[p], c[p]


def get_representation(name):
    if name == 'tfidf':
        return TfidfVectorizer()
    elif name == 'bow':
        return CountVectorizer(min_df=10, stop_words=stopwords.words('portuguese'))


def get_classifier(name):
    if name == 'OCSVM':
        return OneClassSVM(gamma='auto', kernel='poly')
    if name == 'rl':
        return LogisticRegression(class_weight='balanced')


def split_by_tweet(df):
    df_new = pd.concat([pd.Series(row['ids'], row['text'])
                        for _, row in df.iterrows()]).reset_index()

    df_new['label'] = df_new[0].apply(lambda x: 0 if 'C' in x else 1)
    df_new = df_new.rename(columns={0: 'ids', 'label': 'label', 'index': 'text'})
    return df_new


# ao copiar/colar verifique se os caracteres de acentuaçao nas regexs estão corretos
def makeTags(text):
    text = str(text).lower()
    if re.search(r"diagn[oó]stic", text, flags=re.DOTALL):
        return 'diagnostico'
    if re.search(r"tratamento", text, flags=re.DOTALL):
        return 'tratamento'
    if re.search(r"depress[aã]o", text, flags=re.DOTALL):
        return 'depressao'
    if re.search(r"ansiedade", text, flags=re.DOTALL):
        return 'ansiedade'
    if re.search(r"depressivo|tarja\sp|ansiol[íi]tico", text, flags=re.DOTALL):
        return 'antidepress'
    if re.search(r"m[eé]dic[oa]|psic[oó]l[oó]g[oa]|psiquiatra|neurologista|terapeuta", text, flags=re.DOTALL):
        return 'medico'
    return ''


def make_tag_feature(text, list_features):
    text = str(text).lower()
    combined = "(" + ")|(".join(list_features) + ")"
    if re.match(combined, text, flags=re.DOTALL):
        return 'found'
    return ''


# exemplo de uso - marca todos os textos que contêm palavras de natureza clínica
def exemplo(df):
    df['Event'] = ''
    df['Event'] = df.apply(lambda x: [makeTags(t) for t in x['Text']], axis=1)
    return df


def check_existance(df, list_features):
    df['Event'] = ''
    df['Event'] = df.apply(lambda x: [make_tag_feature(t, list_features) for t in x['Text']], axis=1)
    return df


class CustomTextDataset(Dataset):
    def __init__(self, tokens_mask, labels):
        self.tokens_mask = tokens_mask
        self.labels = labels
        self.size = 50

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        label = self.labels[idx]
        data = self.tokens_mask[idx]
        start = random.randint(0, data['tokens'].shape[0] - self.size)
        data_short = {v: data[v][start: start + self.size] for v in data}
        return [data_short, label]



In [ ]:

import numpy as np
import pandas as pd

from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split


class Preprocessing:
	
    def __init__(self, args):
        self.data = 'data/tweets.csv'
        # self.data = 'data/tweets.csv'
        self.max_len = args['max_len']
        self.max_words = args['max_words']
        self.test_size = args['test_size']
        self.task = args['task']

    def unison_shuffled_copies(self, a, b):
        assert len(a) == len(b)
        np.random.seed(42)
        p = np.random.permutation(len(a))
        return a[p], b[p]


    def load_data(self, task):
        text_train, label_train, text_test, label_test = get_data_one_round(task)
        text_train, label_train, text_test, label_test = text_train.to_numpy(), label_train.to_numpy(), text_test.to_numpy(), label_test.to_numpy()
        label_train[label_train == -1] = 0
        label_test[label_test == -1] = 0
        max_post = 200
        np.random.seed(42)
        # shuffle and select posts
        text_train, label_train = unison_shuffled_copies(text_train, label_train)
        for i in range(text_train.shape[0]):
            text_train[i] = '\n'.join(np.random.choice(text_train[i], min(len(text_train[i]), max_post), replace=False))

        text_test, label_test = unison_shuffled_copies(text_test, label_test)
        for i in range(text_test.shape[0]):
            text_test[i] = '\n'.join(np.random.choice(text_test[i], min(len(text_test[i]), max_post), replace=False))
        self.x_train, self.x_val, self.y_train, self.y_val = train_test_split(text_train, label_train, test_size=0.1, shuffle=True)

        self.x_test, self.y_test = text_test, label_test
        # self.x_train, self.x_test, self.y_train, self.y_test = train_test_split(X, Y, test_size=self.test_size)
        
    def prepare_tokens(self):
        self.tokens = Tokenizer(num_words=self.max_words)
        self.tokens.fit_on_texts(self.x_train)

    def sequence_to_token(self, x):
        sequences = self.tokens.texts_to_sequences(x)
        return sequence.pad_sequences(sequences, maxlen=self.max_len)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class TweetClassifier(nn.ModuleList):
    def __init__(self, args):
        super(TweetClassifier, self).__init__()
        
        self.batch_size = args['batch_size']
        self.hidden_dim = args['hidden_dim']
        self.LSTM_layers = args['lstm_layers']
        self.input_size = args['max_words'] # embedding dimention
        
        self.dropout = nn.Dropout(0.1)
        self.embedding = nn.Embedding(self.input_size, self.hidden_dim, padding_idx=0)
        self.lstm = nn.LSTM(input_size=self.hidden_dim, hidden_size=self.hidden_dim,
                  num_layers=self.LSTM_layers, batch_first=True)
        self.fc1 = nn.Linear(in_features=self.hidden_dim, out_features=100)
        self.fc2 = nn.Linear(100 , 2)
        
    def forward(self, x):
    
        h = torch.zeros((self.LSTM_layers, x.size(0), self.hidden_dim))
        c = torch.zeros((self.LSTM_layers, x.size(0), self.hidden_dim))
        
        torch.nn.init.xavier_normal_(h)
        torch.nn.init.xavier_normal_(c)

        out = self.embedding(x)
        out, (hidden, cell) = self.lstm(out, (h,c))
        out = self.dropout(out)
        out = torch.relu(self.fc1(out[:,-1,:]))
        out = self.dropout(out)
        out = torch.softmax(self.fc2(out), 1)

        return out

In [ ]:
import pickle
import time

import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from collections import Counter

from sklearn.utils import class_weight

from sklearn.metrics import classification_report, f1_score
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data.sampler import WeightedRandomSampler, RandomSampler
from torch.nn import CrossEntropyLoss
from datetime import datetime


class DatasetMaper(Dataset):
    '''
    Handles batches of dataset
    '''

    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]


class Execute:
    '''
    Class for execution. Initializes the preprocessing as well as the
    Tweet Classifier model
    '''

    def __init__(self, args):
        print(f'Training  on task {args["task"]}')
        self.__init_data__(args)

        self.args = args
        self.batch_size = args['batch_size']

        self.model = TweetClassifier(args)
        # self.model.to(device)

    def __init_data__(self, args):
        '''
        Initialize preprocessing from raw dataset to dataset split into training and testing
        Training and test datasets are index strings that refer to tokens
        '''
        self.preprocessing = Preprocessing(args)
        self.preprocessing.load_data(args['task'])
        self.preprocessing.prepare_tokens()

        raw_x_train = self.preprocessing.x_train
        raw_x_test = self.preprocessing.x_test
        raw_x_val = self.preprocessing.x_val

        self.y_train = self.preprocessing.y_train
        self.y_test = self.preprocessing.y_test
        self.y_val = self.preprocessing.y_val

        self.x_train = self.preprocessing.sequence_to_token(raw_x_train)
        self.x_test = self.preprocessing.sequence_to_token(raw_x_test)
        self.x_val = self.preprocessing.sequence_to_token(raw_x_val)

    def train(self, test=False, path=''):

        # Let there be 9 samples and 1 sample in class 0 and 1 respectively
        class_counts = list(Counter(self.y_train).values())
        num_samples = sum(class_counts)
        labels = self.y_train  # corresponding labels of samples

        class_weights = [num_samples / class_counts[i] for i in range(len(class_counts))]
        weights = [class_weights[labels[i]] for i in range(int(num_samples))]
        sampler = WeightedRandomSampler(torch.DoubleTensor(weights), int(num_samples))

        training_set = DatasetMaper(self.x_train, self.y_train)
        sampler = RandomSampler(training_set)
        validation_set = DatasetMaper(self.x_val, self.y_val)
        test_set = DatasetMaper(self.x_test, self.y_test)

        self.loader_training = DataLoader(training_set, batch_size=self.batch_size, sampler=sampler)
        self.loader_validation = DataLoader(validation_set)
        self.loader_test = DataLoader(test_set)
        if test:
            output_structure = {'pred': None, 'true': None}

            # self.model.load_state_dict(torch.load(path))
            self.model = torch.load(f'{PATH_CORPUS}/lstm/{path}')
            val_test = self.evaluation(self.loader_test)
            pred = np.array(val_test).argmax(1)
            output_structure['pred'] = pred
            output_structure['true'] = self.y_test
            print(f'Testing report on task {task}')
            print(classification_report(self.y_test, pred))
            pickle.dump(output_structure,
                        open(f'{PATH_CORPUS}/lstm/predictions/{task}-LSTM.pkl', 'wb'))
            return 

        # print(device)

        optimizer = optim.RMSprop(self.model.parameters(), lr=args['learning_rate'])

        def create_weight_dict(labels):
            unique_labels = np.unique(labels)
            class_weights = class_weight.compute_class_weight(
                                  'balanced', classes=unique_labels, y=labels)
            return class_weights
        vect_weights = create_weight_dict(self.y_train)
        loss_fct = CrossEntropyLoss(weight=torch.tensor(vect_weights, dtype=torch.float32))

        print(f'{datetime.fromtimestamp(time.time())}\t' + 'Start Training')
        for epoch in range(args['epochs']):
            predictions = []

            self.model.train()
            print('Total of ', len(self.loader_training))
            for i, batch in enumerate(self.loader_training):
                # batch = tuple(t.to(device) for t in batch)
                if i % 10 ==0:
                    print(f'{datetime.fromtimestamp(time.time())}\t it {i}')
                x_batch, y_batch = batch
                x = x_batch.type(torch.LongTensor)
                y = y_batch.type(torch.LongTensor).view(-1, 1)

                y_pred = self.model(x)

                loss = loss_fct(y_pred, y.view(-1))


                loss.backward()

                optimizer.step()
                optimizer.zero_grad()

                predictions += list(y_pred.squeeze().detach().numpy())

            val_predictions = self.evaluation(self.loader_validation)
            pred = np.array(val_predictions).argmax(1)
            # train_accuary = self.calculate_accuray(self.y_train, predictions)
            # test_accuracy = self.calculate_accuray(self.y_test, val_predictions)
            print(f'Validation report on task {task} and epoch {epoch}')
            print(
                f'{datetime.fromtimestamp(time.time())}\t' + "Epoch: %d, loss: %.5f" % (epoch + 1, loss.item()))
            w = [class_weights[i] for i in self.y_val]
            print(classification_report(self.y_val, pred))
            print(f'Weighted report')
            print(classification_report(self.y_val, pred, sample_weight=w))
            # print(f'Average Validation f1 {f1_score(self.y_val, pred)}')
            torch.save(self.model, f'{PATH_CORPUS}/lstm/models/{task}-dim50-epoch:{epoch}.pt')

    def evaluation(self, loader):
        predictions = []
        self.model.eval()
        with torch.no_grad():
            for x_batch, y_batch in loader:
                x = x_batch.type(torch.LongTensor)
                y = y_batch.type(torch.FloatTensor)

                y_pred = self.model(x)
                predictions += list(y_pred.detach().numpy())

        return predictions

    @staticmethod
    def calculate_accuray(grand_truth, predictions):
        true_positives = 0
        true_negatives = 0

        for true, pred in zip(grand_truth, predictions):
            if (pred > 0.5) and (true == 1):
                true_positives += 1
            elif (pred < 0.5) and (true == 0):
                true_negatives += 1
            else:
                pass

        return (true_positives + true_negatives) / len(grand_truth)


In [ ]:

def parameter_parser():  
    parser = {}
    parser["epochs"] = 15
    parser["learning_rate"] = 0.001
    parser["hidden_dim"] = 50
    parser["lstm_layers"] = 2
    parser["batch_size"] = 128
    parser["test_size"] = 0.2
    parser["max_len"] = 4000
    parser["max_words"] = 10_000
    parser["task"] = 'depress'
    return parser


In [ ]:

if __name__ == "__main__":
    problems = ['anxiety', 'depress']
    dict_best_model = {'anxiety': 'models/anxiety-dim50-epoch:10.pt',
                       'depress': 'models/depress-dim50-epoch:10.pt'}
    for task in problems:
        print('Start')
        args = parameter_parser()
        print('Got')
        args['task'] = task
        execute = Execute(args)
        execute.train(True, dict_best_model[task])


Start
Got
Training  on task anxiety
Testing report on task anxiety
              precision    recall  f1-score   support

           0       0.88      0.79      0.83      3108
           1       0.13      0.23      0.17       444

    accuracy                           0.72      3552
   macro avg       0.50      0.51      0.50      3552
weighted avg       0.78      0.72      0.75      3552

Start
Got
Training  on task depress
Testing report on task depress
              precision    recall  f1-score   support

           0       0.89      0.82      0.85      2359
           1       0.17      0.26      0.20       337

    accuracy                           0.75      2696
   macro avg       0.53      0.54      0.53      2696
weighted avg       0.80      0.75      0.77      2696

